In [1]:
from pathlib import Path
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from textblob import TextBlob
from datetime import datetime
import os
import sys
import csv
import re
import stanza
import pandas as pd
import numpy as np
import plotly

# Basic paths
src_path = str(Path.cwd().parent / "pdfs")
sys.path.append(src_path)
src_path = str(Path.cwd().parent / "src")
sys.path.append(src_path)

project_path = Path().resolve().parent
csv_path = project_path / "speeches_csv"
sa_path = project_path / "sentiment_analysis" #changed name

# Taking 1 speech per topic

In [40]:
df = pd.read_csv(sa_path / "topic_sa.csv")
df[df['main_topic_1'] == 'Topic 4']

# 'Prayer_Breakfast_2016' 0 
# 'Elena_Kagan_USSC_Nomination' 44
# 'NW_Flight_253_Update' 17
# 'Umpqua_Community_College_Shootings' 3 
# 'White_House_Correspondent_Dinner_2013' 4
# 'Senate_Floor_Immigration_Reform' 6
# 'Border_Security_and_Immigration_Reform' 5
# 'Security_Team_Announcement' 1
# 'UN_Copenhagen_Climate_Change' 8 
# 'India_Speech_at_New_Delhi' 10

[0, 44, 17, 3, 4, 6, 5, 1, 8, 10]

,title,date_x,stanza,textblob,vader,subjectivity,pages,date_y,location,highest_speaker_count,...,main_topic_6,main_topic_6_perc,main_topic_7,main_topic_7_perc,main_topic_8,main_topic_8_perc,main_topic_9,main_topic_9_perc,main_topic_10,main_topic_10_perc
3,Umpqua_Community_College_Shootings,2015-10-01,-0.250,0.061,-0.998,0.435,4,2015-10-01,Washington D.C.,0,...,Topic 1,0.027939,Topic 5,0.027939,Topic 6,0.027939,Topic 10,0.027938,Topic 9,0.027937
15,Aurora_Tragedy,2012-07-20,0.022,0.131,0.998,0.481,3,2012-07-20,"Harborside Event Center, Fort Myers, Florida",0,...,Topic 2,0.055878,Topic 3,0.055877,Topic 10,0.055876,Topic 7,0.055875,Topic 9,0.055873
23,Gun_Violence_Exec_Order_23,2013-01-16,-0.233,0.139,-0.992,0.505,6,2013-01-16,unknown_location,0,...,Topic 5,0.013444,Topic 7,0.013444,Topic 2,0.013444,Topic 6,0.013444,Topic 10,0.013444
28,Senate_Vote_Background_Check,2013-04-17,-0.270,0.060,-0.998,0.393,4,2013-04-17,"White House, Washington D.C.",0,...,Topic 9,0.020165,Topic 3,0.020165,Topic 5,0.020165,Topic 10,0.020164,Topic 6,0.020164
38,Sandy_Hook_Elementary_School,2012-12-14,-0.182,0.179,0.997,0.502,2,2012-12-14,unknown_location,0,...,Topic 3,0.076346,Topic 2,0.076346,Topic 10,0.076345,Topic 7,0.076344,Topic 9,0.076342
39,Newtown_Vigil_Address,NaN,NaN,NaN,NaN,NaN,5,2012-12-16,"Newtown, Connecticut",1,...,Topic 6,0.025451,Topic 10,0.025450,Topic 2,0.025450,Topic 3,0.025449,Topic 9,0.025449
63,Navy_Yard_Shooting_Memorial,2013-09-22,-0.119,0.128,0.999,0.510,5,2013-09-22,"The Marine Barracks, Washington D.C.",0,...,Topic 3,0.016398,Topic 2,0.016397,Topic 10,0.016397,Topic 6,0.016397,Topic 9,0.016397
104,Gun_Violence_Reduction_Steps,NaN,NaN,NaN,NaN,NaN,8,2015-01-05,"White House, Washington D.C.",3,...,Topic 8,0.008638,Topic 2,0.008638,Topic 10,0.008637,Topic 9,0.008637,Topic 3,0.008637
142,Gun_Violence_Denver,2013-04-03,-0.203,0.128,-0.998,0.493,7,2013-04-03,"Denver Police Academy, Denver. Colorado",0,...,Topic 7,0.011418,Topic 6,0.011417,Topic 3,0.011417,Topic 10,0.011417,Topic 9,0.011417
180,Orlando_Shootings_Statement,2016-06-12,-0.060,0.050,0.312,0.570,3,2016-06-12,"White House, Washington D.C.",0,...,Topic 5,0.048790,Topic 6,0.048789,Topic 10,0.048788,Topic 9,0.048788,Topic 7,0.048787


In [7]:
sa_topic = df.groupby('main_topic_1').mean()
sa_topic = sa_topic.reindex(['Topic 1','Topic 2','Topic 3','Topic 4','Topic 5','Topic 6','Topic 7','Topic 8', 'Topic 9', 'Topic 10']) 
sa_topic

,stanza,textblob,vader,subjectivity,pages,highest_speaker_count,count_commas,main_topic_1_perc,main_topic_2_perc,main_topic_3_perc,main_topic_4_perc,main_topic_5_perc,main_topic_6_perc,main_topic_7_perc,main_topic_8_perc,main_topic_9_perc,main_topic_10_perc
main_topic_1,,,,,,,,,,,,,,,,,
Topic 1,-0.079457,0.140609,0.768565,0.476957,7.019608,0.137255,0.901961,50.524537,28.458202,11.884472,6.613988,1.803442,0.526793,0.109548,0.040335,0.019342,0.019341
Topic 2,-0.089000,0.163900,0.911100,0.495500,5.090909,0.090909,0.636364,67.479383,23.035060,6.366554,2.924849,0.032361,0.032360,0.032359,0.032359,0.032358,0.032357
Topic 3,-0.123692,0.118192,0.429962,0.425923,4.344828,0.172414,0.724138,66.974072,21.800014,6.952530,3.195858,0.944571,0.026592,0.026592,0.026591,0.026590,0.026590
Topic 4,-0.147250,0.112750,0.112000,0.482583,5.071429,0.285714,0.785714,68.909521,17.570980,8.397654,4.696651,0.280858,0.028868,0.028868,0.028867,0.028867,0.028866
Topic 5,-0.089389,0.152944,0.992097,0.465528,6.776596,0.351064,0.851064,62.771451,24.090191,8.018768,3.309108,1.113337,0.338063,0.170717,0.115196,0.053639,0.019530
Topic 6,-0.066810,0.157143,0.998905,0.453667,5.269231,0.384615,0.846154,64.089971,22.051291,8.751268,3.186285,1.294493,0.416308,0.089134,0.067179,0.027036,0.027035
Topic 7,-0.216680,0.133880,0.995240,0.455800,6.793103,0.172414,0.586207,72.369624,21.497405,4.723370,0.847818,0.310230,0.191003,0.015138,0.015138,0.015137,0.015137
Topic 8,-0.013620,0.149060,0.937320,0.473900,5.218182,0.127273,0.927273,73.817778,16.727022,6.078433,2.058617,0.794881,0.385445,0.071003,0.022274,0.022274,0.022273
Topic 9,-0.149154,0.128385,0.999538,0.449154,6.684211,0.315789,0.736842,71.252549,16.295338,7.477971,3.588961,0.930725,0.241170,0.114898,0.071236,0.013576,0.013575


In [29]:
samples = df.loc[[], ]

samples = df.loc[[

samples


KeyError: "None of [Index(['Prayer_Breakfast_2016', 'Elena_Kagan_USSC_Nomination',\n       'NW_Flight_253_Update', 'Umpqua_Community_College_Shootings',\n       'White_House_Correspondent_Dinner_2013',\n       'Senate_Floor_Immigration_Reform',\n       'Border_Security_and_Immigration_Reform', 'Security_Team_Announcement',\n       'UN_Copenhagen_Climate_Change', 'India_Speech_at_New_Delhi'],\n      dtype='object')] are in the [index]"